# **Microsoft Stock Price Data Analytics**

## Introduction
In today's fast-paced and ever-changing world of finance, investing in the stock market can be a daunting task, especially with the added uncertainty brought on by events like the COVID-19 pandemic. Many investors find it challenging to make informed decisions, particularly when it comes to choosing stocks like Microsoft. The problem our team aims to tackle is how to provide investors with better insights and tools for making investment decisions in this volatile market.

Why do we want to solve this problem? We understand that investing can be a path to financial security and growth, and we want to empower investors with the right tools and information. Microsoft, being a significant player in the tech industry, attracts considerable attention from investors. By developing a machine learning model to predict Microsoft's stock price, we hope to assist investors in making more informed decisions, reduce uncertainty, and potentially increase their chances of financial success.

Our goal is to harness the power of data and cutting-edge technology to create a predictive model that can analyze historical stock data, market trends, and other relevant factors. This model will provide insights into Microsoft's stock price movements, offering investors a valuable resource to make smarter investment choices in uncertain times. Ultimately, our team is dedicated to using our skills to provide a solution that can benefit both novice and experienced investors in their quest for financial stability and growth in the stock market.

In [ ]:
# declare a list tasks whose products you want to use as inputs
upstream = None


In [1]:
import duckdb 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from os import environ

## ELT Pipeline

Before we can build a predictive model of Microsoft's stock data, we need to access all the relevant information to the stock's price action. Lucky for us, a developer named [@ranaroussi](https://github.com/ranaroussi) and other generous contributors have developed an open-source Python package known as `yFinance`, which allows users to access historical stock market data for many companies in the major stock market indices, such as the NASDAQ or the S&P 500.

To access this rich and dynamic data source, we've built an **Extract, Load, and Transform (ELT) Pipeline** to seamlessly move data from the source ("extract" if you will), "load" it in a data storage tool which, in this case, is the MotherDuck cloud warehouse, and conduct our "transformations," analysis, and modelling in toolsets such as this.

We conducted the *extraction* process through Python scripts within this repository, such as the `initialize_load.py` and `extract_stock_data.py`, which, in short, allowed us to pull data straight from the `yfinance` API and into an in-memory DuckDB database initially. From creating our DuckDB database, which we saved locally as `stock_data.duckdb`, we attached our database file to the MotherDuck cloud warehouse to collaborate openly on this project.

Since we can access the cloud-stored dataset from the line of Python code, `con = duckdb.connect(f"md:?motherduck_token={token}")`, we can consider our process a success as we're now able to apply the data cleaning, transformations, visualizations, and algorithms we need to build a successful prediction model. To gain a high-level perspective on our project's architecture, we've included the diagram below to illustrate our process:
<img src="../../images/elt-pipeline-architecture.png" >

In [3]:
# load_dotenv('../../.env')
# token = os.getenv('motherduck_token')

None


In [4]:
from os import environ

token = environ.get("TOKEN", None)

In [ ]:
# connect to the database
con = duckdb.connect(f"md:?motherduck_token={token}")

# define your SQL query
sql_query = "SELECT * FROM stocks_clouddb.msft_data"

# execute the query and fetch the result into a DataFrame
df = con.sql("SELECT * FROM stocks_clouddb.msft_data").fetchdf().copy()

In [ ]:
# display the first few rows of the dataset
print("First few rows of the dataset:")
print(df.head())

In [ ]:
# get a summary of the dataset
print("Dataset summary:")
print(df.info())

In [ ]:
# summary statistics for numerical columns
print("Summary statistics for numerical columns:")
print(df.describe())

In [ ]:
# check for missing values
print("Missing values:")
print(df.isnull().sum())

In [ ]:
# check for duplicate rows
print("Duplicate rows:")
print(df.duplicated().sum())

## Data Visualization

In the following section, we've created different plots to understand how Microsoft's stock price changed over time before developing an ML model to predict its stock price. We'll be answering the following questions below while doing exploratory analysis:

1. What is the statistical distribution of Microsoft's stock price over the given time period?
2. How has Microsoft's stock price changed over the observed time frame?
3. What is the annual trend in Microsoft's stock price, and how has it evolved over the years?
4. What is the daily volatility of Microsoft's stock price?
5. What is the degree of correlation between Microsoft's stock price and other relevant variables?

### Stock Price Distribution
In this analysis, we generated a histogram with a kernel density estimate (KDE) overlay to examine the distribution of Microsoft's closing stock prices. The resulting chart reveals a positively skewed distribution, where the majority of data points are concentrated on the left side, while the right tail exhibits a long tail extending to higher values. This distribution provides insights into the typical range of closing stock prices and the presence of potential outliers or extreme values. The visualization aids in understanding the underlying statistical characteristics of the dataset, which is crucial for modeling and risk assessment in financial analysis.

In [ ]:
# assuming you've already loaded your dataset into the 'df' DataFrame
# to create a histogram for the "close_price" column

# step 1: Extract the "close_price" column
close_price_data = df['close_price']

# step 2: Create the histogram
plt.figure(figsize=(10, 6))  # Optional: Set the figure size
plt.hist(close_price_data, bins=20, edgecolor='black', color='skyblue')

# step 3: Customize the plot
plt.title('Histogram of Close Prices')
plt.xlabel('Close Price')
plt.ylabel('Frequency')

# step 4: Show the histogram
plt.show()

### Stock Price Changes Over Time
This line chart tracks the historical closing stock prices of Microsoft over the past 35 years. The chart illustrates a period of relative price stability between 2000 and 2015, with the stock trading around the \\$50 mark. However, a significant upward trend emerged after 2015, leading to a remarkable surge in Microsoft's stock value, reaching nearly $350 by 2020. Subsequently, there was a noticeable decline to around \\$200 in 2022, followed by a swift rebound, surpassing the previous peak and exceeding \\$350 in 2023. This chart provides valuable insights into the stock's historical performance, indicating substantial growth potential and noteworthy price fluctuations.

In [ ]:
# line chart of closing stock price over time 
plt.figure(figsize=(10,6))
sns.lineplot(x='datetime', y ='close_price', data = df)
plt.title('Closing Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Closing Stock Price')
plt.show()

### Stock Price Changes by Year
This box plot provides a visual representation of Microsoft's closing stock prices distributed across multiple years. The plot reveals a general trend of increasing stock prices over the years, underscoring the stock's growth potential. Additionally, there are a few notable outliers, which may represent exceptional price movements or events that warrant further investigation. The visualization effectively summarizes the annual variations in stock prices and highlights the overall positive trajectory, making it a valuable tool for investors and analysts assessing Microsoft's historical performance.

In [ ]:
df['year'] = df['datetime'].dt.year
plt.figure(figsize=(12, 6))
sns.boxplot(x='year', y='close_price', data=df)
plt.title('Closing Stock Prices by Year')
plt.xlabel('Year')
plt.ylabel('Closing Stock Price')
plt.xticks(rotation=45, ha='right')
plt.show()

### Correlations with Stock Price
In this analysis, we've constructed a correlation heatmap to explore the relationships between various stock price-related variables, including the opening price, highest price of the day, lowest price of the day, and closing price. The heatmap visualizes the strength and direction of these correlations. It is evident that the opening and closing prices exhibit a robust positive correlation, while the low and high prices demonstrate a relatively weaker positive correlation. This information is valuable for understanding how different aspects of stock pricing are interrelated, aiding in risk management and investment decision-making.

In [ ]:
# create a heatmap of the correlation between stock prices 
corr = df[['open_price','day_high','day_low','close_price']].corr()
plt.figure(figsize=(8,8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Between Stock Prices')
plt.show()

# it shows the opening and closing prices have a strong positive correlation, while the low and high prices have a weaker positive correlation. 

In [ ]:
plt.figure(figsize = (10,6))
sns.histplot(df['close_price'],kde=True)
plt.title('Distribution of Closing Stock Price')
plt.xlabel('Closing Stock Price')
plt.ylabel('Frequency')
plt.show()

### Daily Return Analysis
To gain a deeper understanding of Microsoft's historical stock performance, we have examined the daily percentage changes in stock prices. This line chart illustrates the daily fluctuations in Microsoft's stock returns over time. Notably, it showcases instances of sudden drops in stock value, often followed by rapid rebounds. Over the last 2,000 days, the stock has demonstrated relative stability in its daily returns, with periodic fluctuations. This analysis of daily returns offers a finer perspective on the stock's volatility and resilience, which is vital for short-term trading strategies and risk assessment.

In [ ]:
# create a line chart of the daily returns over time 
daily_returns = df['close_price'].pct_change()

plt.plot(daily_returns.index, daily_returns.values)

plt.title('Microsoft Daily Return')
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.show()

## Conclusion
The EDA process commenced with data retrieval from Yahoo Finance, storing it in the 'MotherDuck' repository, and establishing a pipeline to ensure regular data updates every Friday. Data cleaning procedures were applied to ensure data integrity and consistency by formatting and eliminating duplicate values.

In addition, we conducted exploratory data analysis by creating different visualizations, including line plots, box plots, histograms, and heatmaps, were employed to extract valuable insights from the dataset. These visualizations unveiled latent trends, patterns, and relationships that may not have been readily discernible from the raw data.

In summary, the visualizations collectively portray Microsoft's stock as having exhibited sustained growth over time, interspersed with periods of volatility. This data-driven analysis lays the foundation for our ongoing efforts, including predictive modeling, enabling us to make well-informed financial decisions and investments.